In [12]:
import pandas as pd
import yfinance as yf

In [16]:
bvsp = yf.download('^BVSP', start='2011-01-01', end='2020-12-31')
bvsp_vol = bvsp['Volume']
bvsp_close = bvsp['Close']
bvsp_returns = bvsp['Close'].pct_change().dropna()

[*********************100%***********************]  1 of 1 completed


C:\Users\loren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:689: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [20]:
# BVSP RSI Index
# RSI = 100 - (100 / (1 + RS))
# RS = Average Gain / Average Loss
# Average Gain = Sum of gains over the past 14 periods / 14
# Average Loss = Sum of losses over the past 14 periods / 14

change = bvsp_returns

change_up = change.copy()
change_down = change.copy()

change_up[change_up<0] = 0
change_down[change_down>0] = 0

# Verify that we did not make any mistakes
print(change.equals(change_up+change_down))

# Calculate the rolling average of average up and average down
avg_up = change_up.rolling(14).mean()
avg_down = change_down.rolling(14).mean().abs()

rsi = 100 * avg_up / (avg_up + avg_down)

rsi

True


Date
2011-01-04          NaN
2011-01-05          NaN
2011-01-06          NaN
2011-01-07          NaN
2011-01-10          NaN
                ...    
2020-12-22    70.244874
2020-12-23    71.139128
2020-12-28    72.056895
2020-12-29    73.299583
2020-12-30    72.674238
Name: Close, Length: 2471, dtype: float64

In [21]:
df = pd.DataFrame({'bvsp_vol': bvsp_vol, 'rsi': rsi})
df.to_csv('bvsp_vol_rsi.csv', index=False)